In [18]:
import pandas as pd
import numpy as np
import joblib
import ta
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import classification_report

import tensorflow as tf

Sequential = tf.keras.models.Sequential
LSTM = tf.keras.layers.LSTM
Dense = tf.keras.layers.Dense
Dropout = tf.keras.layers.Dropout
EarlyStopping = tf.keras.callbacks.EarlyStopping
Input = tf.keras.layers.Input

# Load dataset
df = pd.read_csv("BTCUSDT_1m.csv", index_col="timestamp", parse_dates=True)
# Add technical indicators
df["rsi"] = ta.momentum.RSIIndicator(df["close"]).rsi()
df["ema_10"] = ta.trend.EMAIndicator(df["close"], window=10).ema_indicator()
df["ema_50"] = ta.trend.EMAIndicator(df["close"], window=50).ema_indicator()
macd = ta.trend.MACD(df["close"])
df["macd"] = macd.macd_diff()
bollinger = ta.volatility.BollingerBands(df["close"])
df["bollinger_mavg"] = bollinger.bollinger_mavg()
df["bollinger_hband"] = bollinger.bollinger_hband()
df["bollinger_lband"] = bollinger.bollinger_lband()

# Binary target: 1 = Up, 0 = Not Up (Down or Neutral)
# threshold = 0.0025
df["target"] = (df["close"].shift(-10) - df["close"]) / df["close"]
df["target2"] = (df["close"].shift(-10) - df["close"])
df["target"] = df["target"].apply(lambda x: 1 if x > threshold else 0)
df["target2"] = df["target"].apply(lambda x: 1 if x > 0 else 0)
# # Drop rows with NaNs (after ta indicators and shift)
# df.dropna(inplace=True)

# Select features (exclude 'close' to avoid target leakage)
# features = df[["open", "high", "low", "volume", "rsi", "ema_10", "ema_50", "macd", "bollinger_mavg", "bollinger_hband", "bollinger_lband"]]
# labels = df["target"]

# # Scale features
# scaler = StandardScaler()
# features_scaled = scaler.fit_transform(features)
# joblib.dump(scaler, "scaler.pkl")

# # Create sequences (30 timesteps)
# sequence_length = 30
# X, y = [], []
# for i in range(len(features_scaled) - sequence_length):
#     X.append(features_scaled[i:i + sequence_length])
#     y.append(labels.iloc[i + sequence_length])

# X = np.array(X)
# y = np.array(y)

# # Train/test split (preserve time order)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# # Compute class weights for imbalance
# class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
# class_weights_dict = dict(enumerate(class_weights))
# print("Class weights:", class_weights_dict)

# # Build Binary LSTM model
# model = Sequential([
#     Input(shape=(sequence_length, X.shape[2])),
#     LSTM(64, return_sequences=True),
#     Dropout(0.3),
#     LSTM(64),
#     Dropout(0.3),
#     Dense(32, activation="relu"),
#     Dense(1, activation="sigmoid")
# ])
# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# # Train model with early stopping
# early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
# model.fit(
#     X_train, y_train,
#     epochs=20,
#     batch_size=64,
#     validation_split=0.1,
#     class_weight=class_weights_dict,
#     callbacks=[early_stop],
#     verbose=1
# )

# # Evaluate model
# loss, accuracy = model.evaluate(X_test, y_test)
# print(f"\u2705 Test Accuracy: {accuracy:.4f}")

# # Save model
# model.save("btc_lstm_binary_model.h5")
# print("\u2705 Binary LSTM model saved as btc_lstm_binary_model.h5")

# # Predictions & Classification Report
# y_pred_probs = model.predict(X_test)
# y_pred = (y_pred_probs > 0.5).astype(int).flatten()
# print("\n\ud83d\udcca Classification Report:")
# print(classification_report(y_test, y_pred, target_names=["Not Up", "Up"]))


In [22]:
df.tail(50)

,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_volume,taker_buy_quote_volume,ignore,rsi,ema_10,ema_50,macd,bollinger_mavg,bollinger_hband,bollinger_lband,target,target2
timestamp,,,,,,,,,,,,,,,,,,,,
2025-05-04 12:19:00,95553.20,95553.21,95553.20,95553.20,2.52946,1746361199999,2.416980e+05,152,1.93288,184692.888545,0,65.065057,95525.946866,95474.513523,11.088382,95487.1420,95581.704281,95392.579719,0,0
2025-05-04 12:20:00,95553.21,95553.21,95488.65,95488.65,6.83056,1746361259999,6.524213e+05,1282,3.32419,317468.712281,0,50.159517,95519.165617,95475.067895,5.200550,95490.5540,95580.026235,95401.081765,0,0
2025-05-04 12:21:00,95488.66,95488.66,95488.65,95488.65,1.10351,1746361319999,1.053727e+05,131,0.97158,92774.872283,0,50.159517,95513.617323,95475.600526,1.056117,95492.0570,95580.332877,95403.781123,0,0
2025-05-04 12:22:00,95488.65,95488.66,95460.00,95460.01,3.48594,1746361379999,3.328062e+05,1154,1.89839,181226.589117,0,44.870152,95503.870537,95474.989133,-3.608474,95491.3550,95580.412346,95402.297654,0,0
2025-05-04 12:23:00,95460.01,95460.01,95454.09,95454.10,1.04807,1746361439999,1.000452e+05,269,0.37364,35666.065829,0,43.842734,95494.821349,95474.169951,-6.895765,95490.3580,95580.606878,95400.109122,0,0
2025-05-04 12:24:00,95454.10,95454.10,95442.40,95454.10,4.50044,1746361499999,4.295539e+05,741,3.10579,296438.437648,0,43.842734,95487.417467,95473.382894,-8.754586,95489.3605,95580.742217,95397.978783,0,0
2025-05-04 12:25:00,95454.09,95461.85,95454.09,95461.85,18.95819,1746361559999,1.809774e+06,496,2.01756,192590.649670,0,45.732521,95482.768837,95472.930624,-9.106087,95488.7505,95580.694032,95396.806968,0,0
2025-05-04 12:26:00,95461.84,95461.84,95438.78,95438.79,22.58717,1746361619999,2.156055e+06,939,1.99871,190793.626854,0,41.281085,95474.772685,95471.591776,-10.439871,95487.2870,95581.407331,95393.166669,0,0
2025-05-04 12:27:00,95438.79,95459.00,95429.60,95459.00,21.37040,1746361679999,2.039708e+06,1094,9.21167,879245.896587,0,46.221624,95471.904924,95471.097981,-9.534226,95487.2935,95581.405998,95393.181002,0,0


In [21]:
df.isna().sum()

open                       0
high                       0
low                        0
close                      0
volume                     0
close_time                 0
quote_asset_volume         0
number_of_trades           0
taker_buy_base_volume      0
taker_buy_quote_volume     0
ignore                     0
rsi                       13
ema_10                     9
ema_50                    49
macd                      33
bollinger_mavg            19
bollinger_hband           19
bollinger_lband           19
target                     0
target2                    0
dtype: int64